In [1]:
import pandas as pd
import asyncio
from langchain_community.document_transformers import DoctranTextTranslator
from langchain_core.documents import Document
import openai
import nest_asyncio
from langchain.text_splitter import CharacterTextSplitter

In [2]:
from langchain_elasticsearch import ElasticsearchStore
from langchain_openai import OpenAIEmbeddings
import elasticsearch
import getpass
from functools import partial

In [3]:
nest_asyncio.apply()

In [20]:
df = pd.read_csv('/Users/jessgarson/Docments/tour_press_update.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/jessgarson/Documents/tour_press_update.csv'

In [5]:
df

,title,date,link,text
0,算法锐舞,5/20,https://mp.weixin.qq.com/s/3kbLFsL3-bEEc6XCd_fSsw,时间：6月1日周六，19:00-22:30\n地点：SYSTEM系统（淮海中路1327号云海...
1,2024国际即兴编程大会,5/13,https://mp.weixin.qq.com/s/lFGIMV-BEyND_9zb-z3buA,2024国际即兴编程大会\n2024年5月30日 - 2024年6月1日\n中国上海·上海纽...
2,关于本周五晚上活动的Visceral Realists,5/26,https://mp.weixin.qq.com/s/mbwzNNSYbzsu-Pj8fsSFIQ,\n佐那·赞合罗斯，本能现实主义的生命力来源之一，一名多媒体艺术家，常驻纽约的声音制造组合“...
3,5.30｜老爹的数据轰炸 Daddy’s Data Detonation,5/26,https://mp.weixin.qq.com/s/oT1r0Hif1ms8Hde2Cwb8Iw,是的，老爹又回来了，这次老爹也决定尝试点新玩意儿。他带着来自纽约的噪音组合，利用现场编程来为...
4,这周末 (5.31-6.02),5.31,https://mp.weixin.qq.com/s/6vXpcunONsfrV6vxvwuBAQ,Date 时间: 5.31 8:30PM\nVenue 地点: Yuyintang 育音堂\...


In [6]:
qa_translator = DoctranTextTranslator(language="english", openai_api_model="gpt-4o")

In [7]:
async def translate_text(text):
    document = Document(page_content=text)
    loop = asyncio.get_event_loop()
    response = await loop.run_in_executor(
        None,
        partial(
            openai.chat.completions.create,
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a translator."},
                {"role": "user", "content": f"Translate the following text to English: {document.page_content}"}
            ]
        )
    )
    # print(response)
    translated_text = response.choices[0].message.content.strip()
    return translated_text

In [8]:
async def translate_dataframe(df, column_name):
    tasks = [translate_text(text) for text in df[column_name]]
    translated_texts = await asyncio.gather(*tasks)
    df['translated_text'] = translated_texts

In [9]:
loop = asyncio.get_event_loop()
loop.run_until_complete(translate_dataframe(df, 'text'))

In [10]:
df

,title,date,link,text,translated_text
0,算法锐舞,5/20,https://mp.weixin.qq.com/s/3kbLFsL3-bEEc6XCd_fSsw,时间：6月1日周六，19:00-22:30\n地点：SYSTEM系统（淮海中路1327号云海...,"Time: Saturday, June 1st, 19:00-22:30\nLocatio..."
1,2024国际即兴编程大会,5/13,https://mp.weixin.qq.com/s/lFGIMV-BEyND_9zb-z3buA,2024国际即兴编程大会\n2024年5月30日 - 2024年6月1日\n中国上海·上海纽...,**2024 International Conference on Live Coding...
2,关于本周五晚上活动的Visceral Realists,5/26,https://mp.weixin.qq.com/s/mbwzNNSYbzsu-Pj8fsSFIQ,\n佐那·赞合罗斯，本能现实主义的生命力来源之一，一名多媒体艺术家，常驻纽约的声音制造组合“...,"Zana Zangharos, one of the vital sources of In..."
3,5.30｜老爹的数据轰炸 Daddy’s Data Detonation,5/26,https://mp.weixin.qq.com/s/oT1r0Hif1ms8Hde2Cwb8Iw,是的，老爹又回来了，这次老爹也决定尝试点新玩意儿。他带着来自纽约的噪音组合，利用现场编程来为...,"Here is the translated text:\n\n---\n\nYes, th..."
4,这周末 (5.31-6.02),5.31,https://mp.weixin.qq.com/s/6vXpcunONsfrV6vxvwuBAQ,Date 时间: 5.31 8:30PM\nVenue 地点: Yuyintang 育音堂\...,Date: 5.31 8:30PM\nVenue: Yuyintang\nLineup: Y...


In [11]:
es_client = elasticsearch.Elasticsearch(
    getpass.getpass("Host: "),
    api_key=getpass.getpass("API Key: "),
)

Host:  ········
API Key:  ········


In [12]:
embedding = OpenAIEmbeddings()
elastic_vector_search = ElasticsearchStore(
    index_name="lo_tour_demo",
    es_connection=es_client,
    embedding=embedding,
)

In [13]:
translated_texts = df['translated_text'].tolist()
combined_text = "\n".join(translated_texts)
text_splitter = CharacterTextSplitter(chunk_size=1100, chunk_overlap=0)
docs = text_splitter.split_documents([Document(page_content=combined_text)])

In [14]:
elastic_vector_search.add_documents(docs)

['9de91302-0ae8-48e5-af94-8a2c245f05f4',
 '7eaaea3e-a538-4e4d-b8f1-190b3bfdf81d',
 '17bec54a-b240-448a-bf00-a18218f828a1',
 'e17cba09-e5a8-4ab5-a89c-3e2c8f93590f',
 'e5f9288e-58b1-437d-b50c-0a45319bfe62',
 '92959d94-57da-4384-a906-6fe9c88e8aab',
 '3a9ae3cd-5b23-4a9e-975f-a63d41a23880',
 '223cf0f1-87f4-468d-b909-0e32ad5e68e5',
 '2ba2523e-9939-4be8-a9cb-7564f414cb84',
 '600fdecc-4c98-4288-8aff-86abb33c5e4e',
 '99b187c7-6f01-4d03-ae06-fba2e2e7273b',
 '29058d2d-d329-4b42-a215-2970f49cb75f',
 'dcc513be-6536-4d9a-a464-4ebf6461d49e',
 '21f621c3-a008-49f4-af73-10b7f693bbcf',
 '4e835498-cfb9-4940-b6ae-22b3aed6ca7e',
 'ec08822c-0c8e-42de-8d18-747779e347dc']

In [15]:
query = "Tell me about the International Conference on Live Coding?"

In [16]:
results = elastic_vector_search.similarity_search(query)

In [17]:
print(results)

[Document(page_content='Since 2015, the International Conference on Live Coding (ICLC) has been dedicated to studying and practicing the use of computer code as an element of live performance. Previous conferences held in the UK, Canada, Mexico, and the Netherlands have brought together an international community, presenting key insights into this creative practice from technical, philosophical, educational, and governance perspectives.\n\n**Agenda Highlights**\n\n- First presentation of an international conference in Asia\n- 3 days, 3 venues, 72 hours\n- Lectures/Paper Presentations/Workshops/Concerts/Algorave (algorithmic rave) – five major content formats\n- Over 27 international artists and groups from around the globe\n- More than 30 exciting events throughout the conference\n- We look forward to your participation!\n\nImage\nImage\nImage\n* To view the English version of the agenda and artist information, please scan this QR code ⬇️\nImage\n\n**Event Venues**'), Document(page_con

In [18]:
query_system = "What's happening at System?"

In [19]:
system_results = elastic_vector_search.similarity_search(query_system, k=10)

In [21]:
print(system_results)

[Document(page_content="Time: Saturday, June 1st, 19:00-22:30\nLocation: SYSTEM (3rd floor, Yungai Building, 1327 Huaihai Middle Road)\n\nArtists: Zona Zanjeros, Andy Borsz, Messica Arson, Domenico Cipriani, Alo Allik, Bito, Ivan Dianov, Chunk No Reace, R Tyler, +777000, nunez \n\nThe 2024 International Conference on Improvisational Programming will conclude with a distinctive Algorave music concert. Algorave, a form of live coding performance set to dance music, will be hosted at Shanghai's vibrant cultural hub, SYSTEM."), Document(page_content='Date: 5.31 10PM  \nVenue: SYSTEM  \nLineup: Dadaglobal, Q, DJ Gonno, Tatz, Omen, Bubble, Goga  \nGenre: Electronic  \n\nDate: 5.31 10PM  \nVenue: ALL  \nLineup: RYOTA, DJ CO.KR, BAADAAM, 1STER, CIKA KOO, SILKBOI  \nGenre: Electronic  \n\nDate: 5.31 10PM  \nVenue: Heim  \nLineup: Bugsy, Kensa, How?well, Cice, Jeff Chong  \nGenre: Electronic  \n\nDate: 5.31 10PM  \nVenue: Abyss  \nLineup: ABEM, Jaya, Huizit, Done, XIWI  \nGenre: Electronic'), Do